# Fraud Detection Model Development

**Objective:** Build production-ready anomaly detection models for fraud detection

## Data Loading

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

df = pd.read_csv('transactions.csv')
print(f"Dataset shape: {df.shape}")

print(f"\nFraud rate: {df['is_fraud'].mean():.2%}")
df.head()

C:\Users\91630\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Dataset shape: (100000, 15)

Fraud rate: 2.00%


,transaction_id,customer_id,card_number,timestamp,amount,merchant_id,merchant_category,merchant_lat,merchant_long,is_fraud,fraud_type,hour,day_of_week,month,distance_from_home
0,TXN_00000000,CUST_00861,CARD_18969,2025-08-20T00:23:18Z,3117.77,MERCHANT_255,electronics,22.7890,88.6956,1,account_takeover,0,2,8,36.03
1,TXN_00000001,CUST_03773,CARD_96975,2025-09-13T21:39:57Z,840.86,MERCHANT_212,electronics,13.0835,77.4841,0,none,21,5,9,5.07
2,TXN_00000002,CUST_03093,CARD_39004,2025-09-05T12:25:30Z,4070.50,MERCHANT_577,luxury_goods,18.6067,72.4481,1,merchant_collusion,12,4,9,15.60
3,TXN_00000003,CUST_00467,CARD_83329,2025-06-20T12:58:11Z,1176.75,MERCHANT_472,grocery,28.3219,76.7205,0,none,12,4,6,8.62
4,TXN_00000004,CUST_04427,CARD_75134,2025-07-28T19:38:56Z,306.80,MERCHANT_486,gas,28.3160,76.6884,0,none,19,0,7,15.82


## Feature Engineering

In [2]:
from sklearn.preprocessing import LabelEncoder

def engineer_features(data):
    """Feature engineering pipeline"""
    df_features = data.copy()
    
    # 1. Amount features
    df_features['amount_log'] = np.log1p(df_features['amount'])
    df_features['amount_squared'] = df_features['amount'] ** 2
    
    # 2. Categorical features
    df_features['is_high_risk_category'] = df_features['merchant_category'].isin(
        ['jewelry', 'luxury_goods']
    ).astype(int)
    
    # Encode merchant category
    le = LabelEncoder()
    df_features['merchant_category_encoded'] = le.fit_transform(df_features['merchant_category'])
    
    # 3. Temporal features
    df_features['is_late_night'] = ((df_features['hour'] >= 23) | (df_features['hour'] <= 4)).astype(int)
    df_features['is_weekend'] = (df_features['day_of_week'] >= 5).astype(int)
    df_features['is_business_hours'] = ((df_features['hour'] >= 9) & (df_features['hour'] <= 17)).astype(int)
    
    # 4. Distance features
    df_features['distance_risk'] = (df_features['distance_from_home'] > 50).astype(int)
    df_features['distance_log'] = np.log1p(df_features['distance_from_home'])
    
    # 5. Interaction features
    df_features['amount_distance_interaction'] = df_features['amount'] * df_features['distance_from_home']
    df_features['high_amount_late_night'] = (
        (df_features['amount'] > df_features['amount'].quantile(0.9)) & 
        (df_features['is_late_night'] == 1)
    ).astype(int)
    
    return df_features

df_engineered = engineer_features(df)
print("Feature engineering completed")
print(f"Original features: {df.shape[1]}")
print(f"Engineered features: {df_engineered.shape[1]}")
print(f"\nNew features created:")
new_features = set(df_engineered.columns) - set(df.columns)
print(list(new_features))

Feature engineering completed
Original features: 15
Engineered features: 26

New features created:
['merchant_category_encoded', 'is_late_night', 'distance_risk', 'amount_distance_interaction', 'high_amount_late_night', 'is_high_risk_category', 'amount_log', 'amount_squared', 'distance_log', 'is_weekend', 'is_business_hours']


In [3]:
from sklearn.model_selection import train_test_split

feature_columns = [
    'amount', 'amount_log', 'amount_squared',
    'merchant_category_encoded', 'is_high_risk_category',
    'merchant_lat', 'merchant_long',
    'hour', 'day_of_week', 'month',
    'is_late_night', 'is_weekend', 'is_business_hours',
    'distance_from_home', 'distance_log', 'distance_risk',
    'amount_distance_interaction', 'high_amount_late_night'
]

X = df_engineered[feature_columns]
y = df_engineered['is_fraud']

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Training fraud rate: {y_train.mean():.2%}")
print(f"Test fraud rate: {y_test.mean():.2%}")

Training set: 80000 samples
Test set: 20000 samples
Training fraud rate: 2.00%
Test fraud rate: 2.00%


In [4]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_normal = X_train_scaled[y_train == 0]
print(f"\nNormal transactions for training: {X_train_normal.shape[0]}")

joblib.dump(scaler, 'models/scaler.pkl')
print("Scaler saved successfully")


Normal transactions for training: 78400
Scaler saved successfully


## Model Development

### Approach 1: Isolation Forest

In [5]:
from sklearn.ensemble import IsolationForest

print("Training Isolation Forest...")
iso_forest = IsolationForest(
    contamination=0.02,
    random_state=42,
    n_estimators=100,
    max_samples='auto',
    n_jobs=-1
)

iso_forest.fit(X_train_normal)

# Predictions (returns -1 for outliers, 1 for inliers)
iso_pred_train = iso_forest.predict(X_train_scaled)
iso_pred_test = iso_forest.predict(X_test_scaled)

# Convert to binary (1=fraud, 0=normal)
iso_pred_train_binary = (iso_pred_train == -1).astype(int)
iso_pred_test_binary = (iso_pred_test == -1).astype(int)

# Anomaly scores (higher = more anomalous)
iso_scores_train = -iso_forest.score_samples(X_train_scaled)
iso_scores_test = -iso_forest.score_samples(X_test_scaled)

print("Isolation Forest trained successfully")

Training Isolation Forest...
Isolation Forest trained successfully


### Approach 2: One-Class SVM

In [6]:
from sklearn.svm import OneClassSVM

print("Training One-Class SVM...")
sample_size = min(20000, len(X_train_normal))
sample_indices = np.random.choice(len(X_train_normal), sample_size, replace=False)
X_train_svm = X_train_normal[sample_indices]

oc_svm = OneClassSVM(
    kernel='rbf',
    gamma='scale',
    nu=0.02,
)

oc_svm.fit(X_train_svm)

# Predictions
svm_pred_train = oc_svm.predict(X_train_scaled)
svm_pred_test = oc_svm.predict(X_test_scaled)

# Convert to binary
svm_pred_train_binary = (svm_pred_train == -1).astype(int)
svm_pred_test_binary = (svm_pred_test == -1).astype(int)

# Decision scores
svm_scores_train = -oc_svm.decision_function(X_train_scaled)
svm_scores_test = -oc_svm.decision_function(X_test_scaled)

print("One-Class SVM trained successfully")

Training One-Class SVM...
One-Class SVM trained successfully


### Approach 3: Autoencoder

In [7]:
from tensorflow.keras import layers
from tensorflow import keras

print("Building Autoencoder...")

input_dim = X_train_scaled.shape[1]

# Encoder
encoder_input = layers.Input(shape=(input_dim,))
x = layers.Dense(64, activation='relu')(encoder_input)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(16, activation='relu')(x)
encoded = layers.Dense(8, activation='relu', name='bottleneck')(x)

# Decoder
x = layers.Dense(16, activation='relu')(encoded)
x = layers.BatchNormalization()(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(64, activation='relu')(x)
decoded = layers.Dense(input_dim, activation='linear')(x)

autoencoder = keras.Model(encoder_input, decoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['mae'])

print(autoencoder.summary())

Building Autoencoder...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 18)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           1,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bottleneck (Dense)                   │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 16)                  │             144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 16)                  │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │             544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 18)                  │           1,170 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,506 (33.23 KB)

 Trainable params: 8,218 (32.10 KB)

 Non-trainable params: 288 (1.12 KB)

None


In [8]:
reconstructions_train = autoencoder.predict(X_train_scaled)
reconstructions_test = autoencoder.predict(X_test_scaled)

ae_scores_train = np.mean(np.abs(X_train_scaled - reconstructions_train), axis=1)
ae_scores_test = np.mean(np.abs(X_test_scaled - reconstructions_test), axis=1)

# Determining threshold (95th percentile of normal training data reconstruction error)
threshold = np.percentile(ae_scores_train[y_train == 0], 98)
print(f"Reconstruction error threshold: {threshold:.4f}")

# Predictions based on threshold
ae_pred_train_binary = (ae_scores_train > threshold).astype(int)
ae_pred_test_binary = (ae_scores_test > threshold).astype(int)

print("Autoencoder trained successfully")

2500/2500 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Reconstruction error threshold: 1.3757
Autoencoder trained successfully
